# Curate entity identifiers

To make data queryable by an entity identifier, one needs to ensure that identifiers comply to a chosen standard.

Bionty has {meth}`~bionty.Entity.curate` for this.

In [ ]:
from bionty import Gene, CellMarker, lookup
import pandas as pd

## Curating genes

To illustrate it, take a DataFrame that stores a number of gene identifiers, some of them corrupted.

In [ ]:
data = {
    "gene symbol": ["A1CF", "A1BG", "FANCD1", "corrupted"],
    "hgnc id": ["HGNC:24086", "HGNC:5", "HGNC:1101", "corrupted"],
    "ensembl_gene_id": [
        "ENSG00000148584",
        "ENSG00000121410",
        "ENSG00000188389",
        "corrupted",
    ],
}
df_orig = pd.DataFrame(data).set_index("ensembl_gene_id")

In [ ]:
df_orig

To do this, we need a reference identifier (specified as the `id` parameter when initiating the Gene class), the list can be looked up via {meth}`~bionty.lookup`.

In [ ]:
lookup.gene_id

To curate the DataFrame into queryable form, we create an index that corresponds to a default identifier (By default, we use `ensembl_gene_id`).

The default behavior is to curate the index if a column name is not provided.

In [ ]:
Gene().curate(df_orig)

You may provide a column name to curate a specific column against a reference identifier.

In [ ]:
Gene(id=lookup.gene_id.hgnc_id).curate(df_orig, column="hgnc id")

When mapping symbols, the function will automatically convert the aliases into standardized symbols. In this example, `PD-1` is converted into `PDCD1`.

In [ ]:
Gene(id=lookup.gene_id.symbol).curate(df_orig, column="gene symbol")

## Standardizing antibody targets

In [ ]:
import pandas as pd

In [ ]:
proteins = pd.DataFrame(
    index=["CCR7", "OX40", "CD69", "CD8A", "CD45RA", "CD4", "CD3", "CD137"]
)

In [ ]:
cell_marker = CellMarker()

In [ ]:
cell_marker.curate(proteins)